In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

In [2]:
day_i = pd.read_csv('data/Metro_testB/record_2019-01-26.csv')

In [3]:
day_i.head()

,time,lineID,stationID,deviceID,status,userID,payType
0,2019-01-26 00:00:02,C,55,2644,0,B61190c49383709148037b817380b568d,1
1,2019-01-26 00:00:03,C,43,2004,0,B26531cd9eb17acd0d1bd5dc0ad861ea2,1
2,2019-01-26 00:00:03,B,16,875,0,C978f4c5213bc3d01a8382cc62c5fd49d,2
3,2019-01-26 00:00:03,B,8,403,0,D673ffea0fa703d5823de45077f47c5da,3
4,2019-01-26 00:00:04,B,8,405,0,Bf09db6dca54ee9fb2c530037091a8fca,1


In [4]:
predict = pd.read_csv('data/Metro_testB/testB_submit_2019-01-27.csv')

In [5]:
predict.head()

,stationID,startTime,endTime,inNums,outNums
0,0,2019-01-27 00:00:00,2019-01-27 00:10:00,NaN,NaN
1,0,2019-01-27 00:10:00,2019-01-27 00:20:00,NaN,NaN
2,0,2019-01-27 00:20:00,2019-01-27 00:30:00,NaN,NaN
3,0,2019-01-27 00:30:00,2019-01-27 00:40:00,NaN,NaN
4,0,2019-01-27 00:40:00,2019-01-27 00:50:00,NaN,NaN


In [6]:
19//10

1

In [7]:
day_i['time'] = pd.to_datetime(day_i.time)
day_i['minute_count'] = (day_i['time'].dt.minute+day_i['time'].dt.hour*60)//10

In [8]:
predict['time'] = pd.to_datetime(predict['startTime'])
predict['minute_count'] = (predict['time'].dt.minute+predict['time'].dt.hour*60)//10

In [9]:
predict.head()

,stationID,startTime,endTime,inNums,outNums,time,minute_count
0,0,2019-01-27 00:00:00,2019-01-27 00:10:00,NaN,NaN,2019-01-27 00:00:00,0
1,0,2019-01-27 00:10:00,2019-01-27 00:20:00,NaN,NaN,2019-01-27 00:10:00,1
2,0,2019-01-27 00:20:00,2019-01-27 00:30:00,NaN,NaN,2019-01-27 00:20:00,2
3,0,2019-01-27 00:30:00,2019-01-27 00:40:00,NaN,NaN,2019-01-27 00:30:00,3
4,0,2019-01-27 00:40:00,2019-01-27 00:50:00,NaN,NaN,2019-01-27 00:40:00,4


In [10]:
day_i['status'].value_counts()

1    1123837
0    1121707
Name: status, dtype: int64

In [11]:
out = day_i[day_i.status==0]
come = day_i[day_i.status==1]

In [12]:
come_count = come.groupby(['stationID','minute_count'],as_index=False).agg({'status':'count'})
out_count = out.groupby(['stationID','minute_count'],as_index=False).agg({'status':'count'})

In [13]:
come_count.head()

,stationID,minute_count,status
0,0,33,13
1,0,34,33
2,0,35,31
3,0,36,49
4,0,37,53


In [14]:
predict.head()

,stationID,startTime,endTime,inNums,outNums,time,minute_count
0,0,2019-01-27 00:00:00,2019-01-27 00:10:00,NaN,NaN,2019-01-27 00:00:00,0
1,0,2019-01-27 00:10:00,2019-01-27 00:20:00,NaN,NaN,2019-01-27 00:10:00,1
2,0,2019-01-27 00:20:00,2019-01-27 00:30:00,NaN,NaN,2019-01-27 00:20:00,2
3,0,2019-01-27 00:30:00,2019-01-27 00:40:00,NaN,NaN,2019-01-27 00:30:00,3
4,0,2019-01-27 00:40:00,2019-01-27 00:50:00,NaN,NaN,2019-01-27 00:40:00,4


In [15]:
predict_new = predict.merge(come_count,on=['stationID','minute_count'],how='left')
predict['inNums'] = list(predict_new['status'])
predict_new = predict.merge(out_count,on=['stationID','minute_count'],how='left')
predict['outNums'] = list(predict_new['status'])

In [16]:
predict.shape

(11664, 7)

In [17]:
# come

In [18]:
# temp
come = pd.read_pickle('come.pkl').rename(columns={'minute':'minute_count'})
predict['inNums']  = list(predict.merge(come,on=['stationID','minute_count'])['label'])
out = pd.read_pickle('out.pkl').rename(columns={'minute':'minute_count'})
predict['outNums']  = list(predict.merge(out,on=['stationID','minute_count'])['label'])

In [19]:
predict.fillna(0,inplace=True)

In [20]:
predict[['stationID','startTime','endTime','inNums','outNums']].to_csv('result/testA_submit_2019-01-29_v2.csv',index=False)

In [21]:
predict

,stationID,startTime,endTime,inNums,outNums,time,minute_count
0,0,2019-01-27 00:00:00,2019-01-27 00:10:00,-0.036537,6.335229e-01,2019-01-27 00:00:00,0
1,0,2019-01-27 00:10:00,2019-01-27 00:20:00,-0.009272,8.501295e-01,2019-01-27 00:10:00,1
2,0,2019-01-27 00:20:00,2019-01-27 00:30:00,-0.000146,9.466170e-01,2019-01-27 00:20:00,2
3,0,2019-01-27 00:30:00,2019-01-27 00:40:00,0.001512,1.657815e-01,2019-01-27 00:30:00,3
4,0,2019-01-27 00:40:00,2019-01-27 00:50:00,0.000006,9.747600e-03,2019-01-27 00:40:00,4
5,0,2019-01-27 00:50:00,2019-01-27 01:00:00,0.000006,4.505799e-13,2019-01-27 00:50:00,5
6,0,2019-01-27 01:00:00,2019-01-27 01:10:00,0.000006,4.505799e-13,2019-01-27 01:00:00,6
7,0,2019-01-27 01:10:00,2019-01-27 01:20:00,0.000006,4.505799e-13,2019-01-27 01:10:00,7
8,0,2019-01-27 01:20:00,2019-01-27 01:30:00,0.000006,4.505799e-13,2019-01-27 01:20:00,8
9,0,2019-01-27 01:30:00,2019-01-27 01:40:00,0.000006,4.505799e-13,2019-01-27 01:30:00,9


In [22]:
predict.shape

(11664, 7)